In [1]:
import os
import json
import pandas as pd

In [2]:
folder_path = '중독_1-11'

In [3]:
output_folder = './중독전처리'

In [4]:
import re

cols = ['paragraph_text', 'paragraph_speaker','loss_of_control','craving','lying','tolerance','withdrawal','salience','resource_investment','daily_functioning','social_problems','negative_consequences']
symptom_cols = ['loss_of_control','craving','lying','tolerance','withdrawal','salience','resource_investment','daily_functioning','social_problems','negative_consequences']

def is_survey_or_rating(text):
    survey_keywords = ['네', '아니요', '모름', '없음', '예', '아니오', '있음', '잘 모르겠음']
    rating_pattern = r'^[0-9]+점\\.?$'
    text_clean = str(text).strip()
    if re.match(rating_pattern, text_clean):
        return True
    if text_clean in survey_keywords:
        return True
    return False

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            paragraphs = data['paragraph']
            df = pd.DataFrame(paragraphs)
            df_selected = df[cols]

            # 설문/평가 텍스트 행 제거
            df_selected = df_selected[~df_selected['paragraph_text'].apply(is_survey_or_rating)]

            # 지정 증상 컬럼 중 하나라도 0이 아닌 행만 필터링
            df_filtered = df_selected[(df_selected[symptom_cols] != 0).any(axis=1)]

            output_path = os.path.join(output_folder, filename.replace('.json', '.csv'))
            df_filtered.to_csv(output_path, index=False, encoding='utf-8-sig')
            print(f'{filename}: 완료, {len(df_filtered)}개 행')

        except Exception as e:
            print(f'{filename}: 에러 발생 - {str(e)}')
            continue

label_addiction_10_check_A081.json: 완료, 0개 행
label_addiction_10_check_A082.json: 완료, 11개 행
label_addiction_10_check_A086.json: 완료, 4개 행
label_addiction_10_check_A087.json: 완료, 1개 행
label_addiction_10_check_A115.json: 완료, 16개 행
label_addiction_11_check_A082.json: 완료, 5개 행
label_addiction_11_check_A086.json: 완료, 1개 행
label_addiction_11_check_A087.json: 완료, 3개 행
label_addiction_11_check_A115.json: 완료, 4개 행
label_addiction_1_check_A008.json: 완료, 27개 행
label_addiction_1_check_A009.json: 완료, 37개 행
label_addiction_1_check_A024.json: 완료, 15개 행
label_addiction_1_check_A035.json: 완료, 10개 행
label_addiction_1_check_A040.json: 완료, 27개 행
label_addiction_1_check_A063.json: 완료, 11개 행
label_addiction_1_check_A078.json: 완료, 10개 행
label_addiction_1_check_A081.json: 완료, 5개 행
label_addiction_1_check_A082.json: 완료, 10개 행
label_addiction_1_check_A085.json: 완료, 35개 행
label_addiction_1_check_A086.json: 완료, 5개 행
label_addiction_1_check_A087.json: 완료, 17개 행
label_addiction_1_check_A092.json: 완료, 11개 행
label_addi

In [5]:
import os
import re
import pandas as pd
from konlpy.tag import Okt
from sentence_transformers import SentenceTransformer

# 1. 텍스트 정규화 함수
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text.strip()

# 2. 한국어 토큰화
okt = Okt()
def tokenize(text):
    return okt.morphs(text)

# 3. 벡터 임베딩 모델 로드
embedding_model = SentenceTransformer('sentence-transformers/LaBSE')

# 처리 대상 폴더
folder_path = '중독전처리'  # CSV 파일들이 위치한 폴더 경로

# 결과 저장용 딕셔너리(파일명 -> 문장별 임베딩)
all_embeddings = {}

for file in os.listdir(folder_path):
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)

        # 'paragraph_text' 컬럼 기준 처리
        normalized_texts = df['paragraph_text'].astype(str).apply(normalize_text)
        tokenized_texts = normalized_texts.apply(tokenize).apply(lambda tokens: ' '.join(tokens))

        # 임베딩 생성
        embeddings = embedding_model.encode(tokenized_texts.tolist())

        # 결과 저장
        all_embeddings[file] = embeddings

        print(f'{file} 처리 완료, 문장 수: {len(normalized_texts)}')

# all_embeddings는 각 파일별로 문장 임베딩 배열 저장


c:\Users\SAMSUNG\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


label_addiction_10_check_A081.csv 처리 완료, 문장 수: 0
label_addiction_10_check_A082.csv 처리 완료, 문장 수: 11
label_addiction_10_check_A086.csv 처리 완료, 문장 수: 4
label_addiction_10_check_A087.csv 처리 완료, 문장 수: 1
label_addiction_10_check_A115.csv 처리 완료, 문장 수: 16
label_addiction_11_check_A082.csv 처리 완료, 문장 수: 5
label_addiction_11_check_A086.csv 처리 완료, 문장 수: 1
label_addiction_11_check_A087.csv 처리 완료, 문장 수: 3
label_addiction_11_check_A115.csv 처리 완료, 문장 수: 4
label_addiction_1_check_A008.csv 처리 완료, 문장 수: 27
label_addiction_1_check_A009.csv 처리 완료, 문장 수: 37
label_addiction_1_check_A024.csv 처리 완료, 문장 수: 15
label_addiction_1_check_A035.csv 처리 완료, 문장 수: 10
label_addiction_1_check_A040.csv 처리 완료, 문장 수: 27
label_addiction_1_check_A063.csv 처리 완료, 문장 수: 11
label_addiction_1_check_A078.csv 처리 완료, 문장 수: 10
label_addiction_1_check_A081.csv 처리 완료, 문장 수: 5
label_addiction_1_check_A082.csv 처리 완료, 문장 수: 10
label_addiction_1_check_A085.csv 처리 완료, 문장 수: 35
label_addiction_1_check_A086.csv 처리 완료, 문장 수: 5
label_addiction_1_ch

In [6]:
import pickle

with open('adddata_all_embeddings.pkl', 'wb') as f:
    pickle.dump(all_embeddings, f)
    
with open('adddata_all_embeddings.pkl', 'rb') as f:
    all_embeddings = pickle.load(f)

